# Fine-tune Bart on x-sum

## Libraries and environment preparation

In [ ]:
#Install essential packages
%%capture
!pip install datasets transformers rouge-score nltk wandb
!apt install git-lfs

In [ ]:
#Colab Environment Check for GPU and RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

#GPU check
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Your runtime has 13.6 gigabytes of available RAM

Sun Feb  6 19:08:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                      

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
# Make sure your version of Transformers is at least 4.11.0 
# to run the following code correctly:
import datasets
import transformers
print(transformers.__version__)

4.16.2


In [ ]:
from transformers import AutoTokenizer    
# Huggingface Automodel class
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint = "facebook/bart-base"

In [ ]:
# Import Wandb 
import os
import wandb
API_KEY = '39991c538626bee25c64d4f8a4c3403dd635537c'
os.environ["WANDB_API_KEY"] = API_KEY

## Loading the dataset

In [ ]:
# import dataset
raw_datasets = datasets.load_dataset("xsum")

Downloading:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/954 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

## Preprocessing the data

In [ ]:
# Import tokenizer from model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
# tokenlize inputs into map

max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/205 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11334
    })
})

## Fine-tuning the model

In [ ]:
# Import tokenizer from model checkpoint and print detail
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [ ]:
# data collator: pad the inputs and labels during each batch to save space
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# keep track with wandb
wandb.init(project="BART-sum")

wandb: Currently logged in as: shusunny (use `wandb login --relogin` to force relogin)


Define `Seq2SeqTrainer` to compute the metrics from the predictions, and also do a bit of pre-processing to decode the predictions into texts:

In [ ]:
# Define traing args, batch size and epoch
# batch size max 8 for input length 1024 on Colab Pro

batch_size = 4
epochs = 1
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    load_best_model_at_end="eval_loss",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=2500, 
    save_steps=5000,
    eval_steps=5000,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",
)

In [ ]:
import nltk
import numpy as np
nltk.download('punkt')

metric = datasets.load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

In [ ]:
# Pass into the trainer

train_dataset=tokenized_datasets["train"]
eval_dataset=tokenized_datasets["validation"].select(range(5666))

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, document, summary.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 204045
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 51012
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
5000,2.397400,2.105726,33.796900,12.870100,27.292200,27.274000,19.503000
10000,2.307800,2.052178,34.358500,13.256100,27.913000,27.913400,19.692900
15000,2.240800,2.007942,34.540800,13.638200,28.312900,28.314700,19.504200
20000,2.185700,1.967978,35.272800,14.345800,28.914500,28.910200,19.581400
25000,2.137600,1.930120,36.151000,15.064800,29.751400,29.751100,19.395000
30000,2.109600,1.904812,36.503700,15.510500,30.077500,30.077100,19.547800
35000,2.056000,1.876446,36.852600,15.721000,30.284100,30.277800,19.514300
40000,2.037700,1.845096,37.118100,15.999300,30.651400,30.644300,19.515700
45000,2.004700,1.824728,37.422400,16.233300,30.951600,30.944300,19.490500
50000,1.990800,1.808068,37.727000,16.489900,31.155400,31.140800,19.530700


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, document, summary.
***** Running Evaluation *****
  Num examples = 5666
  Batch size = 4
Saving model checkpoint to bart-base-finetuned-xsum/checkpoint-5000
Configuration saved in bart-base-finetuned-xsum/checkpoint-5000/config.json
Model weights saved in bart-base-finetuned-xsum/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in bart-base-finetuned-xsum/checkpoint-5000/tokenizer_config.json
Special tokens file saved in bart-base-finetuned-xsum/checkpoint-5000/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, document, summary.
***** Running Evaluation *****
  Num examples = 5666
  Batch size = 4
Saving model checkpoint to bart-base-finetuned-xsum/checkpoint-10000
Configuration saved in bart-base-finetu

TrainOutput(global_step=51012, training_loss=2.1599176275695546, metrics={'train_runtime': 21857.5468, 'train_samples_per_second': 9.335, 'train_steps_per_second': 2.334, 'total_flos': 9.41651411710464e+16, 'train_loss': 2.1599176275695546, 'epoch': 1.0})

In [ ]:
wandb.finish()

eval/gen_len,▄█▄▅▁▅▄▄▃▄
eval/loss,█▇▆▅▄▃▃▂▁▁
eval/rouge1,▁▂▂▄▅▆▆▇▇█
eval/rouge2,▁▂▂▄▅▆▇▇██
eval/rougeL,▁▂▃▄▅▆▆▇██
eval/rougeLsum,▁▂▃▄▅▆▆▇██
eval/runtime,▂▃▃▁▃▂▄█▆▇
eval/samples_per_second,▇▆▆█▆▇▅▁▃▂
eval/steps_per_second,▇▆▆█▆▇▅▁▃▂
train/epoch,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███


In [ ]:
!ls -lh

total 1.5G
drwxr-xr-x 5 root root 4.0K Feb  7 01:15 bart-base-finetuned-xsum
-rw-r--r-- 1 root root 1.5G Feb  7 01:26 bart-base-finetuned-xsum.zip
drwx------ 5 root root 4.0K Feb  7 01:27 drive
drwxr-xr-x 1 root root 4.0K Feb  1 14:32 sample_data
drwxr-xr-x 3 root root 4.0K Feb  6 19:15 wandb


In [ ]:
!zip -r /content/bart-base-finetuned-xsum.zip /content/bart-base-finetuned-xsum/checkpoint-50000/

  adding: content/bart-base-finetuned-xsum/checkpoint-50000/ (stored 0%)
  adding: content/bart-base-finetuned-xsum/checkpoint-50000/training_args.bin (deflated 49%)
  adding: content/bart-base-finetuned-xsum/checkpoint-50000/pytorch_model.bin (deflated 8%)
  adding: content/bart-base-finetuned-xsum/checkpoint-50000/tokenizer.json (deflated 72%)
  adding: content/bart-base-finetuned-xsum/checkpoint-50000/tokenizer_config.json (deflated 45%)
  adding: content/bart-base-finetuned-xsum/checkpoint-50000/rng_state.pth (deflated 27%)
  adding: content/bart-base-finetuned-xsum/checkpoint-50000/special_tokens_map.json (deflated 50%)
  adding: content/bart-base-finetuned-xsum/checkpoint-50000/optimizer.pt (deflated 8%)
  adding: content/bart-base-finetuned-xsum/checkpoint-50000/scaler.pt (deflated 55%)
  adding: content/bart-base-finetuned-xsum/checkpoint-50000/scheduler.pt (deflated 49%)
  adding: content/bart-base-finetuned-xsum/checkpoint-50000/config.json (deflated 63%)
  adding: content/ba

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp bart-base-finetuned-xsum.zip '/content/drive/My Drive/weights/'